# List the factories in QDK/Chemistry

This notebook demonstrates how to list the available factories in QDK/Chemistry and their associated methods.

In [5]:
# Import the optional plugins to expose third-party methods
import qdk_chemistry.plugins.pyscf
import qdk_chemistry.plugins.qiskit

# Define helper functions to style output table
from IPython.display import display, HTML
import html

# Warning display function
def display_warning(message, title="Documentation Warning"):
    html_str = f"""
    <div style="
        border: 2px solid #ff9800; 
        background-color: #fff3e0; 
        padding: 12px; 
        border-radius: 6px; 
        margin: 8px 0;
        display: flex;
        align-items: center;
    ">
        <span style="font-size: 18px; margin-right: 10px;">⚠️</span>
        <div>
            <strong style="color: #e65100;">{html.escape(title)}</strong><br>
            <span style="color: #bf360c;">{html.escape(message)}</span>
        </div>
    </div>
    """
    display(HTML(html_str))

def display_html_table(data, headers):
    """Create an HTML table from data."""
    html_str = "<table>\n<thead>\n<tr>\n"
    for header in headers:
        html_str += f"<th>{html.escape(str(header))}</th>\n"
    html_str += "</tr>\n</thead>\n<tbody>\n"
    
    for row in data:
        html_str += "<tr>\n"
        for header in headers:
            value = row.get(header, '')
            # Truncate long descriptions for better display
            if header == "Description" and len(str(value)) > 100:
                value = str(value)[:97] + '...'
            html_str += f"<td>{html.escape(str(value))}</td>\n"
        html_str += "</tr>\n"
    
    html_str += "</tbody>\n</table>"
    display(HTML(html_str))


In [6]:
# Generate the list of available factories, methods, and settings
import inspect
from qdk_chemistry.algorithms import available, create

factory_inventory = []

for factory_name in available():
    for method_name in available(factory_name):
        factory = create(factory_name, method_name)

        docstring = inspect.getdoc(factory)
        docstring = docstring.split("\n")[0] if docstring else ""
        if not docstring or docstring.startswith("Abstract base class"):
            warning_msg = f"No docstring for {factory_name}:{method_name}"
            display_warning(warning_msg)
            docstring = "Description currently under construction."

        settings = factory.settings()
        for key, value in settings.items():
            row = {
                "Factory": factory_name,
                "Method": method_name,
                "Description": docstring,
                "Setting": key,
                "Default": value,
            }
            factory_inventory.append(row)


In [7]:
# Summarize the results

if factory_inventory:
    headers = list(factory_inventory[0].keys())
    display_html_table(factory_inventory, headers)
else:
    print("No factories found.")

Factory,Method,Description,Setting,Default
active_space_selector,pyscf_avas,PySCF-based Active Space Selector for quantum chemistry calculations.,ao_labels,[]
active_space_selector,pyscf_avas,PySCF-based Active Space Selector for quantum chemistry calculations.,canonicalize,False
active_space_selector,pyscf_avas,PySCF-based Active Space Selector for quantum chemistry calculations.,openshell_option,2
active_space_selector,qdk_occupation,QDK occupation-based active space selector.,occupation_threshold,0.1
active_space_selector,qdk_autocas_eos,QDK entropy-based active space selector.,diff_threshold,0.1
active_space_selector,qdk_autocas_eos,QDK entropy-based active space selector.,entropy_threshold,0.14
active_space_selector,qdk_autocas_eos,QDK entropy-based active space selector.,normalize_entropies,True
active_space_selector,qdk_autocas,QDK Automated Complete Active Space (AutoCAS) selector.,entropy_threshold,0.14
active_space_selector,qdk_autocas,QDK Automated Complete Active Space (AutoCAS) selector.,min_plateau_size,10
active_space_selector,qdk_autocas,QDK Automated Complete Active Space (AutoCAS) selector.,normalize_entropies,True
